In [ ]:
 %load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(0, '/root/sagemaker_ml_insights_repo/notebooks/')

In [ ]:
import pyathena
from pyathena.pandas.cursor import PandasCursor
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from fastprogress import progress_bar
from tqdm.notebook import tqdm

In [ ]:
import mlsec.utils as mlsecutils
from mlsec.tsat import *

In [ ]:
mlsecutils.set_pandas_colwidth(500)

In [ ]:
con = pyathena.connect(region_name='us-east-1',
                       work_group='security_lake_insights',
                      cursor_class=PandasCursor).cursor()

In [ ]:
TABLE = 'amazon_security_lake_glue_db_us_east_1.amazon_security_lake_table_us_east_1_sh_findings'

## Query Security Hub Findings data in Security Lake using Athena Connection

In [ ]:
## Pull data and aggregate by Product|Finding and date
## CONVENTION: ALWAYS HAVE "timestamp" and "value" (frequency) fields in the select statement

sql = f"""
select productname || ' | ' || types[1] as finding_type
    ,timestamp
    ,count(*) as value
from 
(
select metadata.product.feature.name as ProductName, 
finding.types as types, 
cast(from_unixtime(time/1000) as timestamp) as timestamp
FROM {TABLE}
WHERE eventHour > '2022010101'
) tmp
where productname in (
    'Inspector',
    'Macie',
    'Firewall Manager',
    'Systems Manager Patch Manager',
    'GuardDuty',
    'IAM Access Analyzer',
    'Health')
group by 1, 2
"""

keys = ['finding_type']

In [ ]:
results = con.execute(sql).as_pandas()

In [ ]:
 results.shape

In [ ]:
results.head(5)

## Split results data into timeseries dataset

In [ ]:
 def df_to_ts(df):
    ts = pd.Series(df['value'])
    ts.index = pd.to_datetime(df['timestamp'])
    ts = ts.resample('D').sum().fillna(0)
    return ts

In [ ]:
 def split_sql_results_into_timeseries(sql_results, keys):
    timeseries = []
    for _, df in sql_results.groupby(keys, as_index=False):
        ts = df_to_ts(df)
        keyvals = {}
        for key in keys:
            keyvals[key] = df.iloc[0][key]
        timeseries.append((keyvals, ts))
#     timeseries = [({split_key: df.iloc[0][split_key]}, df_to_ts(df)) for _, df in results.groupby(keys, as_index=False)]
    return timeseries

In [ ]:
 timeseries = split_sql_results_into_timeseries(results, keys)

In [ ]:
 len(timeseries)

In [ ]:
timeseries = [(key, ts) for key, ts in timeseries if len(ts) > 5]

In [ ]:
len(timeseries)

In [ ]:
 milo_tss = [MiloTimeseries(ts=ts, source_sql=sql, keyvals=keyvals) for keyvals,ts in timeseries]

In [ ]:
 milo_tss[0].keyvals

In [ ]:
 %store milo_tss

In [ ]:
print("source_sql\n", milo_tss[0].source_sql)
print("_wheres\n", milo_tss[0]._wheres)
print("_selects\n", milo_tss[0]._selects)

In [ ]:
for milo_ts in milo_tss:
    milo_ts.plot()
    plt.show()